# Import modules

In [ ]:
import cudf
import nvcategory
import nvtext
import nvstrings

import os
from collections import OrderedDict
import numpy as np
import datetime as dt

# Download the data
If necessary, download the data from my website and unpack.

In [ ]:
data_url = "http://tomdrabas.com/data/seattle_parking/parking_MayJun2019.tar.gz"

data_dir = "./data"
archive_path = f"{data_dir}/parking_MayJun2019.tar.gz"
file_path = f"{data_dir}/parking_MayJun2019.csv"

directory = os.path.exists(data_dir)
archive = os.path.exists(archive_path)
file = os.path.exists(file_path)

if not directory:
    os.mkdir(data_dir)

if not archive:
    import urllib
    urllib.request.urlretrieve(data_url, archive_path)
    
if not file:
    import tarfile
    tf = tarfile.open(archive_path)
    tf.extractall(path=data_dir)

# Read the data

In [ ]:
%%bash -s "$file_path"
head -n 10 $1

In [ ]:
dtypes = OrderedDict([
    ('OccupancyDateTime', 'date'),
    ('PaidOccupancy', 'int64'),
    ('BlockfaceName', 'str'),
    ('SideOfStreet', 'str'),
    ('SourceElementKey', 'int64'),
    ('ParkingTimeLimitCategory', 'int64'),
    ('ParkingSpaceCount', 'int64'),
    ('PaidParkingArea', 'str'),
    ('PaidParkingSubArea', 'str'),
    ('PaidParkingRate', 'int8'),
    ('ParkingCategory', 'str'),
    ('Location', 'str'),
    ('dow', 'int8')
])

df = cudf.read_csv(
    file_path,
    skiprows=1,
    nrows=1000,
    dtype=list(dtypes.values()),
    names=list(dtypes.keys())
)

df = df.fillna(
    {
        'PaidOccupancy': 0,
        'ParkingSpaceCount': 999,
        'PaidParkingSubArea': 'UKN'
    })

In [ ]:
# size of the file
import os
print('Filesize: {0:.2f}GB'.format(os.path.getsize(file_path) / (1024 ** 3)))

In [ ]:
df['PaidOccupancy']     = df['PaidOccupancy'].astype('float64')
df['ParkingSpaceCount'] = df['ParkingSpaceCount'].astype('float64')

In [ ]:
df.dtypes

In [ ]:
print('The dataset has {0} records and {1} columns.'.format(*df.shape))

In [ ]:
print(df.columns)

In [ ]:
df.head().to_pandas()

# Extract date information

In [ ]:
df['year'] = df['OccupancyDateTime']._column.year
df['month'] = df['OccupancyDateTime']._column.month
df['day'] = df['OccupancyDateTime']._column.day

df['hour'] = df['OccupancyDateTime']._column.hour
df['minute'] = df['OccupancyDateTime']._column.minute

df[['OccupancyDateTime','year','month','day','hour', 'minute']].head().to_pandas()

In [ ]:
counts = df.groupby(['year', 'month', 'day']).agg({'OccupancyDateTime': 'count'})
counts

In [ ]:
print('Average number of transactions per day: {0:.0f}'.format(counts['OccupancyDateTime'].mean()))

# All parking locations

In [ ]:
locations = df[['SourceElementKey', 'BlockfaceName', 'SideOfStreet',
       'ParkingTimeLimitCategory', 'ParkingSpaceCount',
       'PaidParkingArea', 'PaidParkingSubArea',
       'ParkingCategory', 'Location']].drop_duplicates()

locations.head().to_pandas()

In [ ]:
print('Number of parking locations in Seattle: {0}'.format(locations.shape[0]))

In [ ]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon.str.stod()

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon.str.stod()
    
locations['longitude'] = extractLon(locations['Location'])
locations['latitude'] = extractLat(locations['Location'])

locations[['Location', 'longitude', 'latitude']].head().to_pandas()

# Average occupancy

In [ ]:
def avgOccupancy(PaidOccupancy, ParkingSpaceCount, AvgOccupancy):
    for i, (paid, available) in enumerate(zip(PaidOccupancy, ParkingSpaceCount)):
        AvgOccupancy[i] = min(1.0, paid / available) # cap it at 100%, sometimes we see more paid occupancy than spaces available
        
df = (
    df[['OccupancyDateTime', 'PaidOccupancy', 'ParkingSpaceCount'
              , 'SourceElementKey', 'BlockfaceName', 'SideOfStreet'
              , 'ParkingTimeLimitCategory', 'ParkingSpaceCount'
              , 'PaidParkingArea', 'PaidParkingSubArea', 'ParkingCategory', 'dow', 'year', 'month'
              , 'day', 'hour', 'minute']]
    .apply_rows(
        avgOccupancy
        , incols=['PaidOccupancy', 'ParkingSpaceCount']
        , outcols={'AvgOccupancy': np.float64}
        , kwargs={}
    )
)
df.head()

In [ ]:
search_date_f = dt.datetime.strptime('2019-05-24T10:00:00', '%Y-%m-%dT%H:%M:%S')
search_date_t = dt.datetime.strptime('2019-05-24T10:59:59', '%Y-%m-%dT%H:%M:%S')
df.query('''SourceElementKey == 35889 and OccupancyDateTime >= @search_date_f and OccupancyDateTime <= @search_date_t'''
).sort_values(by='OccupancyDateTime').head(5).to_pandas()

In [ ]:
def calcMean(AvgOccupancy, ParkingSpaceCount, MeanOccupancy):
    '''
        Calculate mean
    '''
    for i, (avgOccSum, avgCnt) in enumerate(zip(AvgOccupancy, ParkingSpaceCount)):
        MeanOccupancy[i] = float(avgOccSum) / avgCnt

df_agg_dt = (
    df
    .groupby(['SourceElementKey', 'dow','hour'])
    .agg({
          'ParkingSpaceCount': 'count'
        , 'AvgOccupancy': 'sum'
    })
    .reset_index()
)

df_agg_dt = df_agg_dt.apply_rows(
    calcMean
    , incols=['AvgOccupancy', 'ParkingSpaceCount']
    , outcols={'MeanOccupancy':np.float64}
    , kwargs={}
)

df_agg_dt.drop_column('AvgOccupancy')
df_agg_dt.drop_column('ParkingSpaceCount')

df_agg_dt.head().to_pandas()

# Find the best parking

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="todrabas_test")
location = geolocator.geocode("2300 7th Ave Seattle")

locations['LON_Ref'] = location.longitude
locations['LAT_Ref'] = location.latitude

In [ ]:
from math import sin, cos, sqrt, atan2, pi

def calculateDistance(latitude, longitude, LAT_Ref, LON_Ref, Distance):
    R = 3958.8 # Earth's radius in miles
    
    for i, (lt, ln, lt_r, ln_r) in enumerate(zip(latitude, longitude, LAT_Ref, LON_Ref)):
        lt_rad = lt / 180.0 * pi
        ln_rad = ln / 180.0 * pi
        
        dlon = (ln_r - ln) / 180.0 * pi
        dlat = (lt_r - lt) / 180.0 * pi
        a = (sin(dlat/2.0))**2 + cos(lt_rad) * cos(lt_rad) * (sin(dlon/2.0))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c
        Distance[i] = distance * 5280 # in feet
        
locations = locations.apply_rows(
    calculateDistance
    , incols=['latitude', 'longitude', 'LAT_Ref', 'LON_Ref']
    , outcols={'Distance':np.float64}
    , kwargs={}
)

locations.head().to_pandas()

In [ ]:
# get only meters within 1000 ft
closest = locations.query('Distance < 1000')

closest = closest.merge(df_agg_dt, how='inner', on=['SourceElementKey']).query('dow == 3 and hour == 13')
closest = closest.sort_values(by='MeanOccupancy')

closest_host = closest[['BlockfaceName', 'SideOfStreet',
       'ParkingTimeLimitCategory', 'ParkingSpaceCount', 'PaidParkingArea',
       'PaidParkingSubArea', 'ParkingCategory', 'Location', 'Distance', 'dow', 'hour', 'MeanOccupancy', 'longitude', 'latitude']].head().to_pandas()
closest_host

# Plot the parking spots on the map

We're using gmaps python package that can be found here: https://github.com/pbugnion/gmaps. Follow the instructions contained within the README.md about how to install the package so the map shows properly in jupyter lab.

In [ ]:
closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')
info_box_template = """
<dl>
<dt>Name</dt><dd>{BlockfaceName}</dd>
<dt>Distance</dt><dd>{Distance:.0f}</dd>
<dt>Occupancy (AVG)</dt><dd>{MeanOccupancy:.3f}</dd>
</dl>
"""

parking_info = [info_box_template.format(**parking) for parking in closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')]

In [ ]:
closest_host.to_dict('records')[0]['latitude']

In [ ]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key="--->>> YOUR KEY <<<---") # Your Google API key, go to https://console.developers.google.com

parking_layer = gmaps.symbol_layer(
    closest_host[['latitude', 'longitude']], fill_color="green", stroke_color="green", scale=3, info_box_content=parking_info
)

destinations_layer = gmaps.symbol_layer(
    [[location.latitude, location.longitude]]
    , info_box_content=['DESTINATION']
    , scale=5
    , fill_color="red"
    , stroke_color="red"
)

parkings = closest_host.to_dict('records')

lines_layer = gmaps.drawing_layer(features=[
    gmaps.Line(
        start= (parking['latitude'], parking['longitude'])
        , end = (location.latitude, location.longitude)
        , stroke_weight=2
        , stroke_color="red"
    )
    for parking in parkings]
)

fig = gmaps.figure(layout={'height': '500px'})
fig.add_layer(parking_layer)
fig.add_layer(destinations_layer)
fig.add_layer(lines_layer)
fig